In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/My_Research/RUSTEYE/dataset_cnn_v2.zip

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 8
EPOCHS = 80

In [3]:
train_dir = '/content/dataset_cnn_v2/train'
val_dir   = '/content/dataset_cnn_v2/valid'
test_dir = '/content/dataset_cnn_v2/test'

# Data augmentation cho train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 945 images belonging to 4 classes.
Found 202 images belonging to 4 classes.
Found 205 images belonging to 4 classes.


In [4]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

# Thêm các layer phía trên
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Compile
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=15, mode='min', restore_best_weights=True)

In [7]:
# Train
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 51s 315ms/step - accuracy: 0.7009 - loss: 1.1190 - val_accuracy: 0.8267 - val_loss: 0.5507
Epoch 2/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 15s 126ms/step - accuracy: 0.8913 - loss: 0.2932 - val_accuracy: 0.8317 - val_loss: 0.5372
Epoch 3/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 15s 128ms/step - accuracy: 0.8943 - loss: 0.2827 - val_accuracy: 0.9406 - val_loss: 0.1999
Epoch 4/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 16s 131ms/step - accuracy: 0.9327 - loss: 0.1467 - val_accuracy: 0.8465 - val_loss: 0.4991
Epoch 5/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 15s 124ms/step - accuracy: 0.9395 - loss: 0.1632 - val_accuracy: 0.9010 - val_loss: 0.3130
Epoch 6/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 15s 125ms/step - accuracy: 0.9596 - loss: 0.1158 - val_accuracy: 0.9455 - val_loss: 0.2186
Epoch 7/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 20s 123ms/step - accuracy: 0.9515 - loss: 0.1420 - val_accuracy: 0.9307 - val_loss: 0.2635
Epoch 8/80
119/119 ━━━━━━━━━━━━━━━━━━━━ 16s 134ms/step - accuracy: 0.9652 - loss: 0

In [8]:
# Load best model
from tensorflow.keras.models import load_model
best_model = load_model('best_model.keras')

In [9]:
# Đánh giá trên test set
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Lấy nhãn class
class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Accuracy theo từng class
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 385ms/step - accuracy: 0.9863 - loss: 0.0865

Test Loss: 0.1174
Test Accuracy: 98.54%
26/26 ━━━━━━━━━━━━━━━━━━━━ 8s 171ms/step
Accuracy: 98.53658536585365
Precision: 98.53725542749932
Recall: 98.53658536585365
F1 Score: 98.53247806470675

Classification Report:
                precision    recall  f1-score   support

 bong-troc-son     0.9808    1.0000    0.9903        51
day-cap-ri-set     0.9821    0.9649    0.9735        57
        ri-set     0.9808    0.9808    0.9808        52
       vet-nut     1.0000    1.0000    1.0000        45

      accuracy                         0.9854       205
     macro avg     0.9859    0.9864    0.9861       205
  weighted avg     0.9854    0.9854    0.9853       205


Accuracy for each class:
bong-troc-son: 1.00
day-cap-ri-set: 0.96
ri-set: 0.98
vet-nut: 1.00
